In [1]:
%load_ext autoreload
%autoreload 2

import sys
import json
from tools import *
from pathlib import Path

/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# https://dumps.wikimedia.org/backup-index.html

In [3]:
windows = [2, 5, 10]
min_count = 10
vector_sizes = [50, 100, 500]
languages = ['bxr', 'myv', 'kv']

In [6]:
# window = windows[1]
# vector_size = vector_sizes[1]
language = languages[2]

In [ ]:
# path = Path("./embeddings/"+str(vector_size)+"/"+str(window))
# wes = list(path.iterdir())
# for we in wes:
#     we.unlink()
# path.rmdir()
path.mkdir(parents=True)

In [8]:
wiki2text("./data/"+language+'wiki.xml.bz2', "./data/"+language+'.txt')

number of articles: 2783


In [9]:
for vector_size in vector_sizes:
    for window in windows:
        print("vector_size:", vector_size, "window:", window)
        pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
        w2v(vector_size, window, min_count, fname=pt+"cbow", fcorpus=language+".txt", sg=0)
        w2v(vector_size, window, min_count, fname=pt+"sg", fcorpus=language+".txt", sg=1)
        ft(vector_size, window, min_count, fname=pt+"ft", fcorpus=language+".txt")

vector_size: 50 window: 2
vector_size: 50 window: 5
vector_size: 50 window: 10
vector_size: 100 window: 2
vector_size: 100 window: 5
vector_size: 100 window: 10
vector_size: 500 window: 2
vector_size: 500 window: 5
vector_size: 500 window: 10


In [5]:
scores_d = {}
for language in languages:
    print(language)
    scores_d[language]={}
    for vector_size in [0]:
        for window in windows:            
            pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
            path = Path(pt)
            wes = list(path.iterdir())
            print(wes)
            vocab = get_vocab(wes)
            print('Vocab',len(vocab))
            for we in wes:
                l = we.stem
                
                if l=='ft':
                    continue
                
                if l not in scores_d[language]:
                    scores_d[language][l] = {}
                    
                if vector_size not in scores_d[language][l]:
                    scores_d[language][l][vector_size] = {}
                    
                print(we)
                scores = score("./evaluation/"+language+".json", we, vocab)
                scores_d[language][l][vector_size][window] = loss(scores).item()

bxr
[PosixPath('embeddings/bxr/0/2/pmi'), PosixPath('embeddings/bxr/0/2/cbow')]
embeddings/bxr/0/2/pmi 6124
embeddings/bxr/0/2/cbow 6124
Vocab 6124
embeddings/bxr/0/2/pmi
embeddings/bxr/0/2/cbow
[PosixPath('embeddings/bxr/0/5/pmi'), PosixPath('embeddings/bxr/0/5/cbow')]
embeddings/bxr/0/5/pmi 6124
embeddings/bxr/0/5/cbow 6124
Vocab 6124
embeddings/bxr/0/5/pmi
embeddings/bxr/0/5/cbow
[PosixPath('embeddings/bxr/0/10/pmi'), PosixPath('embeddings/bxr/0/10/cbow')]
embeddings/bxr/0/10/pmi 6124
embeddings/bxr/0/10/cbow 6124
Vocab 6124
embeddings/bxr/0/10/pmi
embeddings/bxr/0/10/cbow
myv
[PosixPath('embeddings/myv/0/2/pmi'), PosixPath('embeddings/myv/0/2/cbow')]
embeddings/myv/0/2/pmi 6948
embeddings/myv/0/2/cbow 6949
Vocab 6948
embeddings/myv/0/2/pmi
embeddings/myv/0/2/cbow
[PosixPath('embeddings/myv/0/5/pmi'), PosixPath('embeddings/myv/0/5/cbow')]
embeddings/myv/0/5/pmi 6949
embeddings/myv/0/5/cbow 6949
Vocab 6949
embeddings/myv/0/5/pmi
embeddings/myv/0/5/cbow
[PosixPath('embeddings/myv/0/10

In [6]:
scores_d

{'bxr': {'pmi': {0: {2: 0.4696427323902769,
    5: 0.4535146372176189,
    10: 0.4396835478016483}},
  'cbow': {0: {2: 0.408880959488578,
    5: 0.3679865334082729,
    10: 0.343756460327813}}},
 'myv': {'pmi': {0: {2: 0.4457854624213767,
    5: 0.4465883536489776,
    10: 0.4424851505270681}},
  'cbow': {0: {2: 0.38855025764639156,
    5: 0.36536250873817616,
    10: 0.3664554968994025}}},
 'kv': {'pmi': {0: {2: 0.5212478999338197,
    5: 0.5074808945742328,
    10: 0.5024685382851773}},
  'cbow': {0: {2: 0.4744826295017139,
    5: 0.46382190047159405,
    10: 0.44166039901996634}}}}

In [9]:
import sys
for method, scores in scores_d['kv'].items():
    print(method)
    for vector_size, windows in scores.items():
#         print(windows.keys())
        ws = [str(round(a,3)) for a in list(windows.values())]
        print("&",vector_size," & "," & ".join(ws),"\\\\")

pmi
& 0  &  0.521 & 0.507 & 0.502 \\
cbow
& 0  &  0.474 & 0.464 & 0.442 \\


In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec


for language in languages:
    fname = str(Path("./data/"+language+".txt").absolute())
    for vector_size in vector_sizes:
        for window in windows:
            pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
            wef = "/tmp/"+str(vector_size)+str(window)
            !glove/vocab_count -min-count $min_count -verbose 2 < $fname > /tmp/vocab.txt
            !glove/cooccur -memory 4.0 -vocab-file /tmp/vocab.txt -verbose 2 -window-size $window  < $fname > /tmp/cooccurrence
            !glove/shuffle -memory 4.0 -verbose 2 < /tmp/cooccurrence > /tmp/cooccurrence.shuf
            !glove/glove -save-file $wef -threads 8 -input-file /tmp/cooccurrence.shuf -x-max 10 -iter 20 -vector-size $vector_size -binary 0 -vocab-file /tmp/vocab.txt -verbose 2

            glove_file = datapath(wef+".txt")
            tmp_file = datapath(pt+"glove")
            _ = glove2word2vec(glove_file,pt+"glove")

In [ ]:
for we in wes:
    print(we)
    scores = score("./evaluation/"+language+".json", we, vocab)
    print(loss(scores))

In [ ]:

ғаилә атай әсәй ағаһы һеңлеһе
семья папа мама брат сестра

илдә район ауыл ҡала
страна район село деревня

йылға волга  лена
река волга нил лена

теле урыҫ башҡорт татарскийй
язык русский бурятский

дин католик мосолман
религия католики мусульмане

тауҙар урал ҡаф
гора

фәндәре математика астрономия
науки

эсемлектәр һыу сәй һут һөт
напиток вода чай сок молоко простокваша

мал һарыҡ һыйыр аттар
скот овца корова конь

кейем кәпәс елән итек
одежда шапка рукавицы халат сапоги

яраҡ тәрилкә ҡалаҡ түңәрәге
посуда тарелка ложка уружка сковородка

ҡош төйлөгән ҡарлуғастар орел
птица коршун ласточка воробей орел
сустав
 
йөҙҙәре күҙҙәре танау ирен маңлайына
лицо глаза нос губа лоб

тәне аяғы  ҡулында арҡаһына күкрәгенә ҡорһаҡ
тело нога рука спина грудь живот

ҡояш
солнце роса туман град иней ливень

сәскә ҡыҙыл күк йәшел
цвет красный синий зеленый

туҡыма ебәк йөн бәрхәт
ткань шелк шерсть бархат

әҫәр йомаҡтар әйтемдәр әкиәт
книга загадка поговорка былина сказка

йорт ишек ҡапҡа диуар ҡыйыҡ баҫҡыстары
дом дверь ворота стена кровля лестница крыльцо

вкус
вкус сладкий соленый горький
 
мәктәп уҡытыусы уҡыу студенттар
школа учитель ученик студент

йыһан ғаләм йондоҙҙар лун
космос вселенная звезда луна

In [1]:
%load_ext autoreload
%autoreload 2

windows = [2, 5, 10]
min_count = 10
vector_sizes = [50, 100, 500]
languages = ['bxr', 'myv', 'kv']
methods = ['PpmiWeighting', 'EpmiWeighting', 'PlmiWeighting', 'PlogWeighting']

from pmi import *


for language in languages:
    print(language)
    x = []
    with open("./data/"+language+".txt") as file:
        for l in file:
            x.append(l.strip())

    for vector_size in [0]:
        for method in methods:
            for window in windows:
                a = build_pairs(x, window)
                prepare_pmi(a)
                pmi(method, vector_size)
                convert("/tmp/space.dm", "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"+method)

bxr
build_pairs: 2
('before filtering', 1318000)
('before filtering', 482241)
prepare_pmi
pmi: 0
('apply', 'PpmiWeighting')
exporting pmi...
done
build_pairs: 5
('before filtering', 3094791)
('before filtering', 1068610)
prepare_pmi
pmi: 0
Progress...1000000
('apply', 'PpmiWeighting')
exporting pmi...
done
build_pairs: 10
('before filtering', 5601615)
('before filtering', 1786685)
prepare_pmi
pmi: 0
Progress...1000000
('apply', 'PpmiWeighting')
exporting pmi...
done
build_pairs: 2
('before filtering', 1318000)
('before filtering', 482241)
prepare_pmi
pmi: 0
('apply', 'EpmiWeighting')
exporting pmi...
done
build_pairs: 5
('before filtering', 3094791)
('before filtering', 1068610)
prepare_pmi
pmi: 0
Progress...1000000
('apply', 'EpmiWeighting')
exporting pmi...
done
build_pairs: 10
('before filtering', 5601615)
('before filtering', 1786685)
prepare_pmi
pmi: 0
Progress...1000000
('apply', 'EpmiWeighting')
exporting pmi...
done
build_pairs: 2
('before filtering', 1318000)
('before filterin

In [ ]:
Positive Local Mutual Information
# Local Mutual Information (akin to the widely used Log-Likelihood Ratio scheme) (Evert, 2005)
The Statistics of Word Cooccurrences

PlogWeighting - Positive Log Weighting 

Exponential Point-wise Mutual Information